In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
# Standart Libraries

import numpy as np

# Import files

import utilities
from Filter import ExtendedKalmanFilter
from Radar import Radar
from RadarSystem import RadarSystem
from Earth import Earth
from Satellite import Satellite
from SatelliteState import SatelliteState

from config import config


In [77]:
# Initialize Earth
earth = Earth()


# Initialize RadarSystem
# Beijing Aerospace Command and Control Center

BACC =  RadarSystem(200, Earth()) 


# Initialize Satellite
R = config['satellite']['initial_conditions']['distance']
theta = config['satellite']['initial_conditions']['polar_angle']
phi = config['satellite']['initial_conditions']['azimuthal_angle']


angular_vel = 0.0010830807404
tang_vel = angular_vel * R
radial_velocity = 0
angular_velocity = 0


sat_state = SatelliteState(np.array([R, theta, phi]), np.array([0]), np.array([radial_velocity, tang_vel, angular_velocity]), np.array([0]))
tiagong = Satellite(sat_state, 0, earth=earth)

In [78]:
R

6778136

In [79]:
# Initialize the Kalman Filter
# tianhe is the chinese super computer

mean_0 = np.array([earth.re/2, np.pi/2, 1, 1, 1, 1])
cov_0 = np.eye(6)
observation_noise = np.eye(2)

tianhe = ExtendedKalmanFilter(mean_0, cov_0, earth, observation_noise=observation_noise)


In [80]:
n = 10000000
simulation = tiagong.simulate(10000000)

sim_lenght = len(simulation.y[0])

predicted_states_satellite_cord = [mean_0]

In [81]:
current_state_satellite_cord = tiagong.get_position_at_t(0)
current_state_earth_cord = utilities.spherical_to_spherical(current_state_satellite_cord)
noise_states_earth_cord = BACC.try_detect_satellite(current_state_earth_cord, i) # Placeholder function. Change for real one

In [82]:
noise_states_earth_cord[0].pos

array([6.77808587e+06, 1.65829041e+00, 1.57781371e+00])

In [83]:
noise_states_earth_cord

[SatelliteState(),
 SatelliteState(),
 SatelliteState(),
 SatelliteState(),
 SatelliteState(),
 SatelliteState(),
 SatelliteState(),
 SatelliteState()]

In [90]:
for i in range(sim_lenght):

    current_state_satellite_cord = tiagong.get_position_at_t(i)
    current_state_earth_cord = utilities.spherical_to_spherical(current_state_satellite_cord)
    noise_states_earth_cord = BACC.try_detect_satellite(current_state_earth_cord, i) # Placeholder function. Change for real one
    #print(len(noise_states_earth_cord))

    if len(noise_states_earth_cord) > 0:
        for state_earth_cord in noise_states_earth_cord:
            state_satellite_cord = utilities.spherical_to_spherical(state_earth_cord.pos)
            new_state_satellite_cord = tianhe.update(state_satellite_cord[:2])
        
    new_state_satellite_cord = tianhe.forecast()[0][:2]

    predicted_states_satellite_cord += new_state_satellite_cord,



In [89]:
predicted_states_satellite_cord

[array([3.18906830e+06, 1.57079633e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00]),
 array([[ 6.37935005e+06],
        [ 2.07079633e+00],
        [ 6.37934026e+06],
        [ 6.95597747e-01],
        [ 1.50000000e+00],
        [-3.87947131e+04]]),
 array([[ 6.89913883e+06],
        [ 3.53022284e+06],
        [ 7.59599181e+06],
        [ 7.19451941e-01],
        [-1.83752660e+04],
        [-6.29615982e+12]]),
 array([[ 6.09983719e+06],
        [ 3.54162447e+09],
        [ 1.94902659e+15],
        [-8.93742514e+03],
        [-3.14807993e+12],
        [-1.63389197e+29]]),
 array([[ 6.49933044e+10],
        [ 9.74643268e+14],
        [ 6.44110111e+35],
        [-1.57403996e+12],
        [-8.16945983e+28],
        [-1.17238133e+66]]),
 array([[ 4.48712611e+014],
        [ 3.22057881e+035],
        [ 2.99471128e+072],
        [-4.08472992e+028],
        [-5.86190666e+065],
        [-4.16734702e+143]]),
 array([[ 1.10410215e+047],
        [ 1.94877683e+079],
    